##Подготовительные расчеты

In [6]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

In [7]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [9]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 66.3MB/s]
  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 32.5MB/s]
  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 13.6MB/s]
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 61.8MB/s]


##Теперь можно заняться наукой
Сначала проведем все иморты

In [150]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Читаем датасеты

In [151]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

Для начала очень простая но не эффективная регрессия

Возьмем из всего набора данных поле о площади и только по ней будем регрессировать

In [152]:
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, Ridge
from sklearn.metrics import mean_squared_log_error, mean_squared_error, mean_absolute_error

In [153]:
def dump_regression(X_train,X_test,y_train,y_test,model):
  func_model = model
  model.fit(X_train,y_train)
  predict = model.predict(X_test)
  rmse = np.sqrt(mean_squared_log_error(predict,y_test))
  mse = mean_squared_error(predict,y_test)
  mae = mean_absolute_error(predict,y_test)
  print(f"rmse: {rmse}")
  print(f"mse: {mse}")
  print(f"mae: {mae}")

X_train = train.loc[:,['LotArea']][:-200]
X_test = train.loc[:,['LotArea']][-200:]
# Split the targets into training/testing sets
y_train = train.loc[:,['SalePrice']][:-200]
y_test = train.loc[:,['SalePrice']][-200:]
dump_regression(X_train,X_test,y_train,y_test,LinearRegression())

rmse: 0.38320675459542114
mse: 4896727686.412853
mae: 54139.23363951724


Ну что тут можно сказать: Только площадь дома плохо подходит, давайте тогда добавим переменных наверн


Попробуем добавить год постройки

In [154]:
X_train = train.loc[:,['LotArea','YearBuilt']][:-200]
X_test = train.loc[:,['LotArea','YearBuilt']][-200:]
# Split the targets into training/testing sets
y_train = train.loc[:,['SalePrice','YearBuilt']][:-200]
y_test = train.loc[:,['SalePrice','YearBuilt']][-200:]
dump_regression(X_train,X_test,y_train,y_test,LinearRegression())

rmse: 0.24142497915969335
mse: 1828338446.628779
mae: 22508.7438759623


Стало лучше, но давайте еще добавим прежде чем будем менять наш подход

In [155]:
# У кроватей может быть null поэтому заполним все null 0
used_params =['LotArea','YearBuilt','GrLivArea']

X_train = train.loc[:,used_params][:-200]
X_test = train.loc[:,used_params][-200:]
# Split the targets into training/testing sets
y_train = train.loc[:,used_params][:-200]
y_test = train.loc[:,used_params][-200:]
dump_regression(X_train,X_test,y_train,y_test,LinearRegression())

rmse: 1.840694865512545e-15
mse: 1.0373103375356389e-23
mae: 1.258134337452551e-12


Так и не понял хорошо это или нет, но вродь стало лучше. Тогда давайте упакуем и отправим посылку

In [156]:
model = LinearRegression()
model.fit(train[used_params], train['SalePrice'])

prediction = model.predict(test[used_params])
sample_submission.head()
sample_submission['SalePrice']=  prediction
sample_submission.to_csv('submission.csv', index = None)

In [17]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Dump solution"

100% 33.6k/33.6k [00:01<00:00, 17.5kB/s]
Successfully submitted to House Prices: Advanced Regression Techniques

Итоговый Скор у нас 0.23864 Я думаю это не плохо, но можно-ли лучше?
## Давайте поменяем линейную модель?

In [157]:
model = SGDRegressor()
model.fit(train[used_params], train['SalePrice'])

prediction = model.predict(test[used_params])
sample_submission.head()
sample_submission['SalePrice']=  prediction
sample_submission.to_csv('submission.csv', index = None)
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Dump solution"

100% 39.5k/39.5k [00:02<00:00, 17.8kB/s]
Successfully submitted to House Prices: Advanced Regression Techniques

Все стало гораздо хуже((( 12.02373 непонятно что произошло вообще. В общем перед тем как посылать посмотрим на своей выборке

In [18]:
model = Lasso()

In [158]:
# У кроватей может быть null поэтому заполним все null 0
used_params =['LotArea','YearBuilt','GrLivArea']

X_train = train.loc[:,used_params][:-200]
X_test = train.loc[:,used_params][-200:]
# Split the targets into training/testing sets
y_train = train.loc[:,used_params][:-200]
y_test = train.loc[:,used_params][-200:]
models = [Lasso(),Ridge(),LinearRegression()]
## Цирк с конями
for model in models:
  print(model)
  dump_regression(X_train,X_test,y_train,y_test,model)
  print()

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
rmse: 1.1067918435832765e-05
mse: 0.0004615485930182334
mae: 0.010606002721621053

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
rmse: 8.979826716512994e-09
mse: 3.0365310149720015e-10
mae: 8.53603307177764e-06

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
rmse: 1.840694865512545e-15
mse: 1.0373103375356389e-23
mae: 1.258134337452551e-12



Пока лучше всех обычная линейная регрессия, Но я подозреваю, что на самом деле там просто очень большое переобучение, ну и метрики у меня и в реальной жизни очень сильно различаются.



Кажется пора воспользоваться кросс валидацией

In [159]:
# Необходимые импорты
from sklearn.model_selection import KFold

In [160]:
def cv_regression(params,train,model,target='SalePrice'):
  kf = KFold(n_splits=8)
  rmse_list = []
  mse_list = []
  mae_list = []
  for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    X_train, X_test = train.loc[train_index, params], train.loc[test_index, params]
    y_train, y_test = train.loc[train_index, target], train.loc[test_index, target]

    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    
    rmse_list.append(np.sqrt(mean_squared_log_error(predict,y_test)))
    mse_list.append(mean_squared_error(predict,y_test))
    mae_list.append(mean_absolute_error(predict,y_test))
    
  print("Mean rmse for 8-fold: {}".format(str(np.mean(rmse_list))))
  print("std rmse for 8-fold: {}".format(str(np.std(rmse_list))))
  print("Mean mse for 8-fold: {}".format(str(np.mean(mse_list))))
  print("std mse for 8-fold: {}".format(str(np.std(mse_list))))
  print("Mean mae for 8-fold: {}".format(str(np.mean(mae_list))))
  print("std mae for 8-fold: {}".format(str(np.std(mae_list))))

used_params =['LotArea','YearBuilt','GrLivArea']

for model in models:
  print(model)
  cv_regression(used_params,train,model)
  print()

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Mean rmse for 8-fold: 0.22646882110643723
std rmse for 8-fold: 0.014241796693895163
Mean mse for 8-fold: 2186453699.1012664
std mse for 8-fold: 486570731.0612697
Mean mae for 8-fold: 30361.030717636935
std mae for 8-fold: 2013.7162360011077

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
Mean rmse for 8-fold: 0.22646883429196257
std rmse for 8-fold: 0.014241802386711903
Mean mse for 8-fold: 2186453693.1539106
std mse for 8-fold: 486570738.9318498
Mean mae for 8-fold: 30361.031501375535
std mae for 8-fold: 2013.7162049983401

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Mean rmse for 8-fold: 0.22646888885476107
std rmse for 8-fold: 0.014241824064942404
Mean mse for 8-fold: 2

Держу в курсе, что теперь оценка по rmse уже очень близка к той которую дает кегл. Но нам нужен скор получше, Плюс в задании нужно поработать с аутлайерами и регуляризировать данные. Давайте этим и займемся

In [161]:
# Возьмем переменную качества забора
print(set(train['Fence']))
# Как мы видем там категориальный признак с nan ну и это то что нам надо. 
# Вообще я подозреваю что данная метрика будет негативно влиять на качество предсказания ибо есть оч дорогие дома без забора как и дешевые дома с крутым забором
from sklearn.preprocessing import LabelEncoder
train['Fence']=train['Fence'].fillna("NO")
train['FenceLabel'] = LabelEncoder().fit_transform(train['Fence'])

{nan, 'MnPrv', 'GdWo', 'MnWw', 'GdPrv'}


In [162]:
used_params =['LotArea','YearBuilt','GrLivArea','FenceLabel']

for model in models:
  print(model)
  cv_regression(used_params,train,model)
  print()

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Mean rmse for 8-fold: 0.22617735897223534
std rmse for 8-fold: 0.013739913755669037
Mean mse for 8-fold: 2186540054.9768124
std mse for 8-fold: 487712416.9215897
Mean mae for 8-fold: 30369.619240176296
std mae for 8-fold: 2027.225272819109

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
Mean rmse for 8-fold: 0.2261772404982233
std rmse for 8-fold: 0.013740012872615288
Mean mse for 8-fold: 2186538710.9738293
std mse for 8-fold: 487713918.9252845
Mean mae for 8-fold: 30369.60692544043
std mae for 8-fold: 2027.2459641195019

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Mean rmse for 8-fold: 0.22617723511436952
std rmse for 8-fold: 0.013739669826956537
Mean mse for 8-fold: 2186

Так, ну не очень сильно поменялось на самом деле, но зато поборолись с аутлайерами и закодировали категориальные признаки.

Теперь давайте нормализуем данные

In [163]:
from sklearn.preprocessing import MinMaxScaler

In [164]:
#train['SalePrice'] = np.log1p(train['SalePrice'])
#X_real_norm_np = MinMaxScaler().fit_transform(X_dum)
train['SalePriceLog'] = np.log1p(train['SalePrice'])
train['LotAreaLog'] = np.log1p(train['LotArea'])
test['LotAreaLog'] = np.log1p(test['LotArea'])

train['YearBuiltLog'] = np.log1p(train['YearBuilt'])
test['YearBuiltLog'] = np.log1p(test['YearBuilt'])

train['GarageAreaNorm'] = np.log1p(train['GarageArea'])
test['GarageAreaNorm'] = np.log1p(test['GarageArea'])

train['YearRemodAddLog'] = np.log1p(train['YearRemodAdd'])
test['YearRemodAddLog'] = np.log1p(test['YearRemodAdd'])

train['GarageCarsNorm'] = np.log1p(train['GarageCars'])
test['GarageCarsNorm'] = np.log1p(test['GarageCars'])

train['GrLivAreaNorm'] = np.log1p(train['GrLivArea'])
test['GrLivAreaNorm'] = np.log1p(test['GrLivArea'])

used_params = ['LotAreaLog','YearBuiltLog','YearRemodAddLog','GarageCarsNorm','OverallCond','GrLivAreaNorm','GarageAreaNorm']

for model in models:
  print(model)
  cv_regression(used_params, train, model,'SalePriceLog')
  print()

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Mean rmse for 8-fold: 0.03059906101603346
std rmse for 8-fold: 0.002011657059774497
Mean mse for 8-fold: 0.15957346645257242
std mse for 8-fold: 0.02012220555217734
Mean mae for 8-fold: 0.309839092997492
std mae for 8-fold: 0.02290976924582491

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
Mean rmse for 8-fold: 0.01622768316404316
std rmse for 8-fold: 0.001257969429716852
Mean mse for 8-fold: 0.04484315360406674
std mse for 8-fold: 0.006488766078740093
Mean mae for 8-fold: 0.1562666576282143
std mae for 8-fold: 0.008403326234886557

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Mean rmse for 8-fold: 0.013543471062673403
std rmse for 8-fold: 0.0010081231784701296
Mean mse fo

Усредним значения всех трех моделей и опубликуем резульатты на кегл

In [168]:
np.any(np.isnan(test[used_params]))

True

In [181]:
test[used_params] = test[used_params].fillna(0)

for i,model in enumerate(models[1:]):
  model.fit(train[used_params], train['SalePriceLog'])
  prediction = model.predict(test[used_params])
  sample_submission.head()
  sample_submission['SalePrice']=  np.exp(prediction)-1
  sample_submission.to_csv(f'submission{i}.csv', index = None)

In [182]:
sub1 = pd.read_csv('submission0.csv')
sub2 = pd.read_csv('submission1.csv')
sub3 = pd.read_csv('submission2.csv')
result = pd.concat([sub1, sub2, sub3], axis=0).dropna(axis=1).groupby('Id').mean()
result.to_csv(f'submission.csv')

In [183]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "merget predicion"

100% 33.7k/33.7k [00:01<00:00, 28.3kB/s]
Successfully submitted to House Prices: Advanced Regression Techniques

Итоговый скор стал меньше чем просто использовать Linearregression но лучше не использовать Lasso незнаю почему так вышло((